# Loads csv files to postgres

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import langdetect
from tqdm import tqdm
from openai import OpenAI

load_dotenv()
LOCAL_POSTGRES = os.getenv('LOCAL_POSTGRES')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

## Load reviews table

In [3]:
# Database connection
engine = create_engine(LOCAL_POSTGRES)

# Load CSV into DataFrame
df = pd.read_csv("output/southern_sun_capetown_.csv")
df

# import psycopg2
# from psycopg2.extras import execute_values

# conn = engine.raw_connection()
# cursor = conn.cursor()

# # Define SQL with conflict handling
# sql = """
# INSERT INTO reviews (column1, column2, column3) 
# VALUES %s
# ON CONFLICT DO NOTHING
# """

# try:
#     execute_values(cursor, sql, df.values.tolist())
#     conn.commit()
#     print(f'Inserted {cursor.rowcount} rows into the reviews table')
# except Exception as e:
#     print(f"Error inserting data: {e}")
# finally:
#     cursor.close()
#     conn.close()


,hotel_id,hotel_name,source_name,positive_review,negative_review,review_rating,reviewer_name,country,sentiment,reviewer_check_in_date,review_created_date,apartment_type,length_nights_stay,group_type,review_feedback,seen,review_text
0,SOUSUN123,Southern Sun Cape Town,booking.com,"Great service, amazing and helpful staff, supe...",NaN,10.0,Marwan,South Africa,joy,January 2025,"January 12, 2025",Double Room with Two Double Beds - Non-Smoking,4,Family,Dear Marwan\n\nThank you for taking the time t...,False,"Positive: Great service, amazing and helpful s..."
1,SOUSUN123,Southern Sun Cape Town,booking.com,Staff are amazing. They really go the extra mi...,As with older hotel rooms the bathrooms often ...,9.0,Sheree,South Africa,joy,March 2025,"March 17, 2025",Double Room with Two Double Beds - Non-Smoking,2,Family,NaN,False,Positive: Staff are amazing. They really go th...
2,SOUSUN123,Southern Sun Cape Town,booking.com,I love the breakfast,It is becoming too expensive,9.0,Afolabi,Nigeria,neutral,March 2025,"March 13, 2025",Standard Queen Room - Non-Smoking,5,Solo traveler,Dear Valued Guest\nThank you for taking the ti...,False,Positive: I love the breakfast negative: It is...
3,SOUSUN123,Southern Sun Cape Town,booking.com,"Great staff, good location",Bar menu,9.0,Alexandra,United States,joy,March 2025,"March 10, 2025",Standard Queen Room - Smoking,2,Solo traveler,Dear Valued Guest\nThank you for taking the ti...,False,"Positive: Great staff, good location negative:..."
4,SOUSUN123,Southern Sun Cape Town,booking.com,NaN,I didn't like that the light on the 'aircon co...,9.0,Yolanda,South Africa,sadness,March 2025,"March 9, 2025",Double Room with Two Double Beds - Non-Smoking,2,Family,Dear Valued Guest\nThank you for taking the ti...,False,Positive: None negative: I didn't like that th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,SOUSUN123,Southern Sun Cape Town,booking.com,Breakfast and location were good.,The hotel declared on booking they have shuttl...,6.0,Adamou,South Africa,sadness,November 2023,"November 29, 2023",Standard Queen Room - Smoking,4,Couple,Thank you so much for your recent visit to Sou...,False,Positive: Breakfast and location were good. ne...
476,SOUSUN123,Southern Sun Cape Town,booking.com,NaN,The Interior was just not worth the money. \nT...,4.0,Dimpho,South Africa,disgust,October 2023,"November 25, 2023",Standard Queen Room - Non-Smoking,3,Couple,Thank you so much for your visit to Southern S...,False,Positive: None negative: The Interior was just...
477,SOUSUN123,Southern Sun Cape Town,booking.com,great location,"the pillows were too soft, no memory foam pill...",5.0,Nishid,South Africa,neutral,November 2023,"November 20, 2023",Standard Queen Room - Non-Smoking,1,Solo traveler,Thank you so much for your recent visit to Sou...,False,Positive: great location negative: the pillows...
478,SOUSUN123,Southern Sun Cape Town,booking.com,Clean room. Great service. Wonderful breakfast...,NaN,10.0,Dr,United States,joy,November 2023,"November 10, 2023",Double Room with Two Double Beds - Non-Smoking,1,Solo traveler,"Good Day, Thank you for taking the time to par...",False,Positive: Clean room. Great service. Wonderful...


## Convert date strings to date objects

In [ ]:
# convert dates to date objects
df["review_created_date"] = pd.to_datetime(df["review_created_date"], errors="coerce").dt.date

# Function to convert check in date to correct date format
def convert_to_date(date_str):
    month, year = date_str.split()  # Split into month and year
    year = f"20{year}"  # Convert two-digit year to four-digit (assumes 20xx)
    print(f"1 {month} {year}").strftime("%Y-%m-%d")
    return pd.to_datetime(f"1 {month} {year}").strftime("%Y-%m-%d")

# Apply function to the column
df["reviewer_check_in_date"] = df["reviewer_check_in_date"].apply(convert_to_date)

df.head()

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1 January 202025, at position 0

## Convert non-English reviews to English

In [37]:
# Function to detect if a text is non-English
def is_non_english(text):
    try:
        return langdetect.detect(text) != "en"
    except:
        return False  # Default to English if detection fails

# Function to translate text using GPT-3.5
def translate_text(text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful AI that translates text to English."},
                {"role": "user", "content": f"Translate the following text to English:\n\n{text}"}
            ],
            temperature=0.3,
            max_tokens=100,
        )
        # return response["choices"][0]["message"]["content"].strip()
        translation = response.choices[0].message.content.strip()
        # print('Translation: ', translation)
        return translation
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # Return original text in case of an error

# Apply translation only to non-English reviews
tqdm.pandas()  # Enable progress bar

df["review_text"] = df["review_text"].progress_apply(
    lambda x: translate_text(x) if is_non_english(x) else x
)

# df["negative_review"] = df["negative_review"].progress_apply(
#     lambda x: translate_text(x) if is_non_english(x) else x
# )


100%|██████████| 428/428 [01:01<00:00,  7.00it/s] 


In [38]:
df[df['country'] == 'Spain']

,hotel_id,hotel_name,source_name,positive_review,negative_review,review_rating,reviewer_name,country,sentiment,reviewer_check_in_date,review_created_date,apartment_type,length_nights_stay,group_type,review_feedback,seen,review_text
186,MORBAY1,Morro Bay Beach Inn,booking.com,Buena ubicación y atención del personal,Nada,8,Jregatero,Spain,joy,2024-11-01,2024-12-01,King Room,1,Couple,Dear jregatero.\nThank you for your review and...,False,Positive: Good location and staff attention\nN...
233,MORBAY1,Morro Bay Beach Inn,booking.com,Cama cómoda,Toallas sucias de rímel y pelo en la cama que ...,6,Silvia,Spain,disgust,2024-08-01,2024-08-17,Queen Room,1,Couple,Dear Silvia.\nThank you for taking the time t...,False,Positive: Comfortable bed\nNegative: Dirty tow...
387,MORBAY1,Morro Bay Beach Inn,booking.com,NaN,NaN,7,Francisca,Spain,NaN,2024-12-01,2024-12-30,Queen Room,1,Family,NaN,False,NaN
395,MORBAY1,Morro Bay Beach Inn,booking.com,NaN,NaN,8,Eric,Spain,NaN,2024-08-01,2024-08-07,Queen Room,1,Family,NaN,False,NaN


In [ ]:
# checking string length for each string type column so it doesn't give LLM error
for col in ["hotel_name", "reviewer_name", "source_name", "sentiment", "country", "group_type"]:
    max_len = df[col].astype(str).apply(len).max()
    print(f"Max length in column {col}: {max_len}")

Max length in column hotel_name: 20
Max length in column reviewer_name: 12
Max length in column source_name: 11
Max length in column sentiment: 8
Max length in column country: 20
Max length in column group_type: 13


In [40]:
# Append data to the PostgreSQL table
num_rows_inserted = df.to_sql("reviews", engine, if_exists="append", index=False)
print(f'inserted {num_rows_inserted} rows into the reviews table')


inserted 428 rows into the reviews table
